In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_Dividend_Yield.csv
/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_Earnings.csv
/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_P_E_ratio.csv
/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_Revenue.csv
/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_Market_Cap.csv


# Data Load

In [2]:
market_cap_df = pd.read_csv('/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_Market_Cap.csv')
earnings_df = pd.read_csv('/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_Earnings.csv')
revenue_df = pd.read_csv('/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_Revenue.csv')
pe_ratio_df = pd.read_csv('/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_P_E_ratio.csv')
dividend_yield_df = pd.read_csv('/kaggle/input/largest-companies-analysis-worldwide/Companies_ranked_by_Dividend_Yield.csv')

# EDA

In [3]:
print(market_cap_df.head(), market_cap_df.info())
print(earnings_df.head(), earnings_df.info())
print(revenue_df.head(), revenue_df.info())
print(pe_ratio_df.head(), pe_ratio_df.info())
print(dividend_yield_df.head(), dividend_yield_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9912 entries, 0 to 9911
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         9912 non-null   int64  
 1   Name         9912 non-null   object 
 2   Symbol       9911 non-null   object 
 3   marketcap    9912 non-null   float64
 4   price (GBP)  9912 non-null   float64
 5   country      9910 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 464.8+ KB
   Rank               Name Symbol     marketcap  price (GBP)        country
0     1              Apple   AAPL  2.689903e+12   176.919492  United States
1     2          Microsoft   MSFT  2.427170e+12   326.537958  United States
2     3             NVIDIA   NVDA  2.259353e+12    92.105725  United States
3     4  Alphabet (Google)   GOOG  1.549815e+12   126.458691  United States
4     5             Amazon   AMZN  1.506038e+12   143.492307  United States None
<class 'pandas.core.frame.DataFra

In [4]:
print(market_cap_df.isnull().sum())
print(earnings_df.isnull().sum())
print(revenue_df.isnull().sum())
print(pe_ratio_df.isnull().sum())
print(dividend_yield_df.isnull().sum())

Rank           0
Name           0
Symbol         1
marketcap      0
price (GBP)    0
country        2
dtype: int64
Rank            0
Name            0
Symbol          1
earnings_ttm    0
price (GBP)     0
country         2
dtype: int64
Rank           0
Name           0
Symbol         1
revenue_ttm    0
price (GBP)    0
country        2
dtype: int64
Rank            0
Name            0
Symbol          1
pe_ratio_ttm    0
price (GBP)     0
country         2
dtype: int64
Rank                  0
Name                  0
Symbol                1
dividend_yield_ttm    0
price (GBP)           0
country               2
dtype: int64


In [5]:
market_cap_df = market_cap_df.dropna(subset=['Symbol', 'country'])
earnings_df = earnings_df.dropna(subset=['Symbol', 'country'])
revenue_df = revenue_df.dropna(subset=['Symbol', 'country'])
pe_ratio_df = pe_ratio_df.dropna(subset=['Symbol', 'country'])
dividend_yield_df = dividend_yield_df.dropna(subset=['Symbol', 'country'])

In [6]:
print(market_cap_df.duplicated().sum())
print(earnings_df.duplicated().sum())
print(revenue_df.duplicated().sum())
print(pe_ratio_df.duplicated().sum())
print(dividend_yield_df.duplicated().sum())

0
0
0
0
0


In [7]:
print(market_cap_df.describe())
print(earnings_df.describe())
print(revenue_df.describe())
print(pe_ratio_df.describe())
print(dividend_yield_df.describe())

              Rank     marketcap   price (GBP)
count  9909.000000  9.909000e+03   9909.000000
mean   4956.438894  8.744509e+09     53.477109
std    2861.386676  5.791520e+10    359.832438
min       1.000000  0.000000e+00      0.000000
25%    2479.000000  2.170231e+08      2.005959
50%    4957.000000  1.067611e+09      8.552696
75%    7434.000000  4.639298e+09     28.817327
max    9912.000000  2.689903e+12  16549.695004
              Rank  earnings_ttm   price (GBP)
count  9909.000000  9.909000e+03   9909.000000
mean   4956.800383  7.113198e+08     53.477189
std    2861.336482  4.204151e+09    359.832842
min       1.000000 -1.730700e+10      0.000000
25%    2480.000000 -4.528400e+06      2.005959
50%    4957.000000  5.363600e+07      8.552696
75%    7434.000000  3.610000e+08     28.817327
max    9912.000000  2.283780e+11  16549.695004
              Rank   revenue_ttm   price (GBP)
count  9909.000000  9.909000e+03   9909.000000
mean   4956.899889  5.707748e+09     53.477085
std    2861.3

## Visualizing Market Capitalization

In [8]:
top_10_market_cap = market_cap_df.sort_values('marketcap', ascending=False).head(10)
fig = px.bar(top_10_market_cap, x='Name', y='marketcap', 
             title='Top 10 Companies by Market Capitalization',
             labels={'marketcap':'Market Cap (GBP)'}, 
             hover_data=['country', 'Symbol'])

fig.show()

## Comparing Earnings vs Market Cap

In [9]:
merged_df = pd.merge(market_cap_df, earnings_df[['Name', 'earnings_ttm']], on='Name')
fig = px.scatter(merged_df, x='marketcap', y='earnings_ttm', 
                 color='country', hover_name='Name', 
                 title='Market Capitalization vs Earnings',
                 labels={'marketcap':'Market Cap (GBP)', 'earnings_ttm':'Earnings (TTM)'})
fig.show()

## Top Companies by Dividend Yield

In [10]:
top_10_dividend_yield = dividend_yield_df.sort_values('dividend_yield_ttm', ascending=False).head(10)
fig = px.bar(top_10_dividend_yield, y='Name', x='dividend_yield_ttm', 
             orientation='h', title='Top 10 Companies by Dividend Yield',
             labels={'dividend_yield_ttm':'Dividend Yield (%)'}, 
             hover_data=['country', 'Symbol'])

fig.show()

## Revenue Distribution by Country

In [11]:
fig = px.box(revenue_df, x='country', y='revenue_ttm', 
             title='Revenue Distribution by Country',
             labels={'revenue_ttm':'Revenue (TTM)'}, 
             hover_name='Name')

fig.show()

## P/E Ratio Analysis

In [12]:
merged_pe_df = pd.merge(market_cap_df, pe_ratio_df[['Name', 'pe_ratio_ttm']], on='Name')
fig = px.scatter(merged_pe_df, x='pe_ratio_ttm', y='marketcap', 
                 size='marketcap', color='country', 
                 hover_name='Name', size_max=60,
                 title='P/E Ratio vs Market Capitalization',
                 labels={'pe_ratio_ttm':'P/E Ratio', 'marketcap':'Market Cap (GBP)'})

fig.show()

## Top Companies in Different Metrics (Dropdown Filter)

In [13]:
combined_df = pd.merge(market_cap_df, earnings_df[['Name', 'earnings_ttm']], on='Name')
combined_df = pd.merge(combined_df, revenue_df[['Name', 'revenue_ttm']], on='Name')
combined_df = pd.merge(combined_df, pe_ratio_df[['Name', 'pe_ratio_ttm']], on='Name')
combined_df = pd.merge(combined_df, dividend_yield_df[['Name', 'dividend_yield_ttm']], on='Name')

In [14]:
fig = px.bar(combined_df, x='Name', y='marketcap', 
             title='Top Companies in Different Financial Metrics',
             labels={'value':'Financial Metric', 'variable':'Metric'},
             hover_data=['country'])

fig.update_layout(updatemenus=[
    dict(
        buttons=list([
            dict(args=[{'y': [combined_df['marketcap']]}], label="Market Cap", method="update"),
            dict(args=[{'y': [combined_df['earnings_ttm']]}], label="Earnings", method="update"),
            dict(args=[{'y': [combined_df['revenue_ttm']]}], label="Revenue", method="update"),
            dict(args=[{'y': [combined_df['pe_ratio_ttm']]}], label="P/E Ratio", method="update"),
            dict(args=[{'y': [combined_df['dividend_yield_ttm']]}], label="Dividend Yield", method="update"),
        ]),
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
    ),
])

fig.show()

# Predicting Stock Prices

In [15]:
combined_df = pd.merge(market_cap_df, earnings_df[['Name', 'earnings_ttm']], on='Name')
combined_df = pd.merge(combined_df, revenue_df[['Name', 'revenue_ttm']], on='Name')
combined_df = pd.merge(combined_df, pe_ratio_df[['Name', 'pe_ratio_ttm']], on='Name')
combined_df = pd.merge(combined_df, dividend_yield_df[['Name', 'dividend_yield_ttm']], on='Name')

In [16]:
combined_df = combined_df.dropna()

In [17]:
X = combined_df[['marketcap', 'earnings_ttm', 'revenue_ttm', 'pe_ratio_ttm', 'dividend_yield_ttm']]
y = combined_df['price (GBP)']#target

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
param_grid = {
    'n_estimators': [100, 300, 500, 800, 1200],
    'max_depth': [10, 20, 30, 40, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [1.0, 'sqrt', 'log2'],  
    'bootstrap': [True, False]
}
rf = RandomForestRegressor(random_state=42)
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid,
                                   n_iter=50, cv=5, verbose=2, random_state=42, n_jobs=-1)

In [20]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END bootstrap=True, max_depth=30, max_features=1.0, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time= 1.0min
[CV] END bootstrap=False, max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=  11.5s
[CV] END bootstrap=False, max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=  12.0s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=300; total time=   6.1s
[CV] END bootstrap=False, max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   2.9s
[CV] END bootstrap=False, max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   2.9s
[CV] END bootstrap=False, max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=10

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, None],
                                        'max_features': [1.0, 'sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 300, 500, 800,
                                                         1200]},
                   random_state=42, verbose=2)

In [21]:
best_params = random_search.best_params_
print("Best parameters found: ", best_params)

Best parameters found:  {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}


In [22]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

In [23]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 214477.7845690917
